In [1]:
from utils import Model
import random
import numpy as np
import os

/home/egor/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import tensorflow as tf


def setup_tensorflow():
    # Create session
    config = tf.ConfigProto(log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=config)

    # Initialize rng with a deterministic seed
    with sess.graph.as_default():
        tf.set_random_seed(FLAGS.random_seed)

    random.seed(FLAGS.random_seed)
    np.random.seed(FLAGS.random_seed)

    summary_writer = tf.summary.FileWriter(FLAGS.train_dir, sess.graph)

    return sess, summary_writer


def setup_inputs(sess, filenames, image_size=None, capacity_factor=3):

    if image_size is None:
        image_size = FLAGS.sample_size
    
    # Read each JPEG file
    reader = tf.WholeFileReader()
    filename_queue = tf.train.string_input_producer(filenames)
    key, value = reader.read(filename_queue)
    channels = 3
    image = tf.image.decode_jpeg(value, channels=channels, name="dataset_image")
    image.set_shape([64, 64, channels])

    # Crop and other random augmentations
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_saturation(image, .95, 1.05)
    image = tf.image.random_brightness(image, .05)
    image = tf.image.random_contrast(image, .95, 1.05)
    image = tf.reshape(image, [1, 64, 64, 3])
    image = tf.cast(image, tf.float32)/255.0
    
    K = 2
    downsampled = tf.image.resize_area(image, [image_size/K, image_size/K])

    feature = tf.reshape(downsampled, [image_size/K, image_size/K, 3])
    label = tf.reshape(image,       [image_size,   image_size,     3])

    # Using asynchronous queues
    features, labels = tf.train.batch([feature, label],
                                      batch_size=FLAGS.batch_size,
                                      num_threads=4,
                                      capacity=capacity_factor*FLAGS.batch_size,
                                      name='labels_and_features')

    tf.train.start_queue_runners(sess=sess)
    
    return features, labels

In [4]:
tf.reset_default_graph()
sess, summary_writer = setup_tensorflow()

In [5]:
def generator(features, params=None, return_params=False):
    # Upside-down all-convolutional resnet
    
    mapsize = 3

    # See Arxiv 1603.05027
    scope = 'g'
    model = Model(scope, features, params)
    
    for j in range(2):
        model.add_residual_block(256, mapsize=mapsize)

    # Spatial upscale (see http://distill.pub/2016/deconv-checkerboard/)
    # and transposed convolution
    # don't apply second upscaling (we need 2x factor)
    model.add_upscale()
    model.add_batch_norm()
    model.add_relu()
    model.add_conv2d_transpose(256, mapsize=mapsize, stride=1, stddev_factor=1.)
       
    # Finalization a la "all convolutional net"
    
    model.add_conv2d(96, mapsize=mapsize, stride=1, stddev_factor=2.)
    # Worse: model.add_batch_norm()
    model.add_relu()

    model.add_conv2d(96, mapsize=1, stride=1, stddev_factor=2.)
    # Worse: model.add_batch_norm()
    model.add_relu()

    # Last layer is sigmoid with no batch normalization
    model.add_conv2d(3, mapsize=1, stride=1, stddev_factor=1.)
    model.add_sigmoid() # TODO: add tanh
    
    gene_vars = model.params
    model.summarize()
    if return_params:
        return model.get_output(), gene_vars
    return model.get_output()

In [6]:
def discriminator(disc_input, params=None, return_params=False):
    # Fully convolutional model
    mapsize = 3
    layers = [64, 128, 256, 512]

    model = Model('d', 2 * disc_input - 1, params)  # normalize between -1 and 1

    for layer in range(len(layers)):
        nunits = layers[layer]
        stddev_factor = 2.0

        model.add_conv2d(nunits, mapsize=mapsize, stride=2, stddev_factor=stddev_factor)
        model.add_batch_norm()
        model.add_relu()

    # Finalization a la "all convolutional net"
    model.add_conv2d(nunits, mapsize=mapsize, stride=1, stddev_factor=stddev_factor)
    model.add_batch_norm()
    model.add_relu()

    model.add_conv2d(nunits, mapsize=1, stride=1, stddev_factor=stddev_factor)
    model.add_batch_norm()
    model.add_relu()

    # Linearly map to real/fake and return average score
    # (softmax will be applied later)
    model.add_conv2d(1, mapsize=1, stride=1, stddev_factor=stddev_factor)
    model.add_mean()

    disc_vars = model.params
    model.summarize()
    if return_params:
        return model.get_output(), disc_vars
    return model.get_output()

In [ ]:
filenames = os.listdir('dataset')
features, labels = setup_inputs(sess, filenames)

In [1]:
# check whether gpu is alive
import tensorflow as tf
tf.reset_default_graph()
sess = tf.InteractiveSession()
with tf.device('gpu:0'):
    a = tf.constant(1, tf.float32, [100, 100])
    b = tf.constant(0.5, tf.float32, [100, 10])
    c = tf.matmul(a, b)
sess.run(c)

/home/egor/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


array([[50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50.

In [ ]:
gen_out, pars = generator(tf.constant(0.0, shape=features.shape, dtype=tf.float32), return_params=True)

In [1]:
# Configuration (alphabetically)
import tensorflow as tf
tf.app.flags.DEFINE_integer('batch_size', 64,
                            "Number of samples per batch.")

tf.app.flags.DEFINE_string('checkpoint_dir', 'checkpoint',
                           "Output folder where checkpoints are dumped.")

tf.app.flags.DEFINE_integer('checkpoint_period', 10000,
                            "Number of batches in between checkpoints")

tf.app.flags.DEFINE_string('dataset', 'dataset',
                           "Path to the dataset directory.")

tf.app.flags.DEFINE_float('epsilon', 1e-8,
                          "Fuzz term to avoid numerical instability")

tf.app.flags.DEFINE_string('run', 'demo',
                            "Which operation to run. [demo|train]")

tf.app.flags.DEFINE_float('gene_l1_factor', .90,
                          "Multiplier for generator L1 loss term")

tf.app.flags.DEFINE_float('learning_beta1', 0.5,
                          "Beta1 parameter used for AdamOptimizer")

tf.app.flags.DEFINE_float('learning_rate_start', 0.00020,
                          "Starting learning rate used for AdamOptimizer")

tf.app.flags.DEFINE_integer('learning_rate_half_life', 5000,
                            "Number of batches until learning rate is halved")

tf.app.flags.DEFINE_bool('log_device_placement', False,
                         "Log the device where variables are placed.")

tf.app.flags.DEFINE_integer('sample_size', 64,
                            "Image sample size in pixels. Range [64,128]")

tf.app.flags.DEFINE_integer('summary_period', 200,
                            "Number of batches between summary data dumps")


tf.app.flags.DEFINE_integer('random_seed', 0,
                            "Seed used to initialize rng.")

tf.app.flags.DEFINE_integer('test_vectors', 16,
                            """Number of features to use for testing""")
                            
tf.app.flags.DEFINE_string('train_dir', 'train',
                           "Output folder where training logs are dumped.")

tf.app.flags.DEFINE_integer('train_time', 20,
                            "Time in minutes to train the model")

FLAGS = tf.app.flags.FLAGS

/home/egor/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
%%time
from arch import BSRGAN
tf.reset_default_graph()
bsrgan = BSRGAN(labels, features, 100000, J=2, J_d=2, M=2, batch_size=FLAGS.batch_size)

In [9]:
tf.trainable_variables()

[<tf.Variable 'g_h0_0000_0000/weight:0' shape=(1, 1, 3, 256) dtype=float32_ref>,
 <tf.Variable 'g_h0_0000_0000/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'g_h1_0000_0000/batch_normalization/beta:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'g_h1_0000_0000/weight:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'g_h1_0000_0000/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'g_h2_0000_0000/batch_normalization/beta:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'g_h2_0000_0000/weight:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'g_h2_0000_0000/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'g_h3_0000_0000/batch_normalization/beta:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'g_h3_0000_0000/weight:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'g_h3_0000_0000/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'g_h4_0000_0000/batch_normalization/beta:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'g_h4_00